- competition/dataset : [https://www.kaggle.com/moltean/fruits](https://www.kaggle.com/moltean/fruits)
- date : 2021/02/15
- original : [https://www.kaggle.com/amadeus1996/fruits-360-transfer-learning-using-keras/](https://www.kaggle.com/amadeus1996/fruits-360-transfer-learning-using-keras/)

## Fruits-360 - Transfer Learning using Keras

**✏ 필사 1회** 

- 이 노트북은 Fruits-360 데이터셋에 대한 transfer leraning을 간략하게 적용했습니다.
- 데이터 셋은 64개의 과일에 대한 42345개의 이미지입니다.
- 우리는 transfer lerning 접근 방법을 일반적인 접근 방법과 비교합니다.
- 2 epochs 이내에 98.44%의 정확도를 달성합니다.

**Contents:**  
1. Brief Explanation of Transfer Learning
2. Transfer Learning using Kaggle Kernels
3. Reading and Visualizing the Data
4. Building and Compiling the Models
5. Training and Validating the Pretained Model
6. Training and Validating Vanilla Model
7. Comparison between Pretained Models and Vanila Model

In [1]:
import os
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import tensorflow as tf

## 1. Brief Explanation of Transfer Learning
transfer learning에서 우리는 먼저 기본 데이터 셋과 작업에 대해 기본 네트워크를 훈련한 다음 학습된 feature를 변경하거나 두 번째 target 네트워크로 전송하여 target 데이터셋과 작업에 대해 훈련합니다. 이 프로세스는 feature가 기본 작업에만 특정하지 않고 기본과 target 작업 모두에 적합할 때 작동하는 경향이 있습니다.  

Lisa Torrey와 Jude Shavlik는 transfer learning을 사용했을 때 가능한 세 가지 이점을 말했습니다:  
- 시작점은 소스 모델의 초기 기술(모델 다듬기 전)이 그렇지 않은 경우보다 높습니다.
- 소스 모델의 훈련 중 스킬의 향상 속도(기울기)는 그렇지 않은 경우보다 가파릅니다.
- 훈련 모델의 수렴은 그렇지 앟은 경우보다 우수합니다.

![transfer learning vs. without transfer](data/Three-ways-in-which-transfer-might-improve-learning.png)  

일반적으로 우리의 데이터셋에 대해 이미 학습과 정교한 튜닝이 된 모델을 사용합니다(네트워크의 weight와 parameter가 큰 데이터셋에 대해 이미 훈련됨). 이미 학습된 모델은 초기화된 가중치를 제공하여 빠른 수렴을 유도하거나 진행중인 작업의 고정된 feature 추출기로 작용할 것이라는 생각입니다.  

transfer learning의 두 가지 주요 시나리오는 다음과 같습니다:  
- convnet 미세 조정: 무작위 초기화 대신 imagenet 1000과 같은 큰 데이터셋으로 훈련된 사전 훈련된 네트워크로 네트워크를 초기화합니다. 나머지 훈련은 일반적입니다. 이 시나리오에서 전체 네트워크는 우리의 관심인 데이터셋에 대해 재훈련시켜야 합니다.
- 고정된 feature 추출기로서의 convnet: 마지막 fully connected layer을 제외한 모든 네트워크에 대해 weight을 고정시킬 것입니다. 마지막 fully connected layer는 random weight을 가진 새로운 것과 교체되며 이 layer만 훈련됩니다.  

이 노트북에서 우리는 첫 번째 시나리오를 보여줄 것입니다.

## 2. Transfer Learning using Kaggle Kernels

### Using the Kears Pretrained Models dataset
Kaggle 커널에서는 네트워크 연결을 사용하여 미리 훈련된 keras 모델을 다운로드할 수 없습니다. [이 데이터셋](https://www.kaggle.com/moltean/fruits)이 kaggle 환경에서 우리가 선호하는 사전 훈련된 모델을 사용하는데 도움이 될 것입니다.  

우리가 해야 할 일은 미리 훈련된 모델을 keras가 찾고 있는 캐시 디렉토리(~/.keras/models)에 복사하는 것입니다.

In [8]:
cache_dir = expanduser(join('~', '.keras'))
if not exists(cache_dir):
    makedirs(cache_dir)
models_dir = join(cache_dir, 'models')
if not exists(models_dir):
    makedirs(models_dir)

In [15]:
!copy G:\GitHub\kaggle\"6. Fruits"\keras-pretrained-models\*notop* C:\Users\PC\.keras\models
!copy G:\GitHub\kaggle\"6. Fruits"\keras-pretrained-models\imagenet_class_index.json C:\Users\PC\.keras\models
!copy G:\GitHub\kaggle\"6. Fruits"\keras-pretrained-models\resnet50* C:\Users\PC\.keras\models

        1개 파일이 복사되었습니다.
G:\GitHub\kaggle\6. Fruits\keras-pretrained-models\resnet50_weights_tf_dim_ordering_tf_kernels.h5
G:\GitHub\kaggle\6. Fruits\keras-pretrained-models\resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
        2개 파일이 복사되었습니다.


In [17]:
print('Available Pretrained Models:\n')
!dir C:\Users\PC\.keras\models

Available Pretrained Models:

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 7AA4-5BDC

 C:\Users\PC\.keras\models 디렉터리

2021-02-15  오후 12:00    <DIR>          .
2021-02-15  오후 12:00    <DIR>          ..
2019-09-21  오전 05:16            35,363 imagenet_class_index.json
2019-09-21  오전 05:16       219,055,592 inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5
2019-09-21  오전 05:17        87,910,968 inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
2019-09-21  오전 05:17       102,853,048 resnet50_weights_tf_dim_ordering_tf_kernels.h5
2019-09-21  오전 05:17        94,653,016 resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
2019-09-21  오전 05:18        58,889,256 vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
2019-09-21  오전 05:18        83,683,744 xception_weights_tf_dim_ordering_tf_kernels_notop.h5
               7개 파일         647,080,987 바이트
               2개 디렉터리  16,929,521,664 바이트 남음


## 3. Reading and Visualizing the Data

### Reading the Data
다른 keras와 같이 이미지 확대 API는 간단하지만 강력합니다. **ImageDataGenerator**를 사용하여 데이터를 패치하고 우리의 네트워크에 넣어보겠습니다.  

keras는 이미지 데이터를 준비하고 확대하기 위한 형태의 **ImageDataGenerator**를 제공합니다. 